In [1]:
#Results were produced in stints. This is the number of the last stint.
run = 1

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from scipy import stats
from scipy import optimize
import joblib

#folder for saving results
filepath = ".../Resultate_final/Put/IS/250/Put_GlueVaR_IS_250_sim_saved/"

In [3]:
#Market and option parameters as in section 4.1 of 'Assessing Asset-Liability Risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
s_0 = 100
r = 0.01
mu = 0.05
sigma = 0.2
tau = 1/52
T = 1/3
K = 100

#Risk measure parameteres
alpha_Glue = 0.95
beta_Glue = 0.995
omega_Glue = np.array([1/3,1/3])

In [4]:
#Sizes for training set, validation set, test set, and set size for Monte Carlo estimation of the risk measures
M_1 = 1500000
M_2 = 500000
M_3 = 500000
M_MC = 500000
#size of the set of data points used to calculate an IS density
M_IS = 250000
#quantile for which the IS density will be computed
alpha_IS = 0.975

In [5]:
#Function for calculating simulated values of S_tau and simulated payoffs P_T from simulations of standard normal random variables
def data_gen(Z,V):
    #simulate S_tau under P
    S_tau = s_0 * np.exp( (mu-0.5*sigma**2)*tau + sigma*np.sqrt(tau)*Z)
    #simulate S_T given S_tau under Q
    S_T = S_tau * np.exp( (r-0.5*sigma**2)*(T-tau) + sigma*np.sqrt(T-tau)*V)
    #calculate corresponding discounted payoffs
    P_T = np.exp(-r*(T-tau)) * np.maximum(K-S_T,0)
    return S_tau, P_T

#The density function of Z
def f(y):
    return stats.norm.pdf(y, loc=0, scale=1)

#The density function of Z_\theta (note that \theta is replaced by x; this is needed for the least-squares solver to work)
def f_theta(x, y):
    return stats.norm.pdf(y, loc=x[0], scale=x[1])

#This function describes the approximation of the expression inside the sum of m_2(theta)
def g_q_alpha_hat_reweighted(x,L,q_alpha_hat):
    return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)

#bounds for the IS density parameters (for the mean parameter no bound is necessary, the standard deviation however needs to be non-negative)
bnds_lower = np.array([-np.inf,0])
bnds_upper = np.array([np.inf,np.inf])

#function for calculating GlueVaR in an IS setting
def GlueVaR_IS(omega, L, alpha, beta, w):
    j_beta = 0
    w_sum_tmp = 0
    while(w_sum_tmp <= (1-beta)):
        w_sum_tmp += w[j_beta]
        j_beta += 1
        
    j_alpha = j_beta
    while(w_sum_tmp <= (1-alpha)):
        w_sum_tmp += w[j_alpha]
        j_alpha += 1
        
    ES_beta = 1/(1-beta) * np.sum(w[0:j_beta-1]*L[0:j_beta-1]) + ( 1 - (1 / (1-beta)) * np.sum(w[0:j_beta-1]) )*L[j_beta]
    ES_alpha = 1/(1-alpha) * np.sum(w[0:j_alpha-1]*L[0:j_alpha-1]) + ( 1 - (1 / (1-alpha)) * np.sum(w[0:j_alpha-1]) )*L[j_alpha]
    VaR_alpha = L[j_alpha]

    return omega[0]*ES_beta + omega[1]*ES_alpha + (1-omega[0]-omega[1])*VaR_alpha

In [6]:
for j in range(100):
    #Generating realisations of standard normal random variables
    Z_train = np.random.normal(loc=0, scale=1, size=M_1)
    V_train = np.random.normal(loc=0, scale=1, size=M_1)
    Z_val = np.random.normal(loc=0, scale=1, size=M_2)
    V_val = np.random.normal(loc=0, scale=1, size=M_2)
    Z_test = np.random.normal(loc=0, scale=1, size=M_3)
    V_test = np.random.normal(loc=0, scale=1, size=M_3)
    Z_MC = np.random.normal(loc=0, scale=1, size=M_MC)
    V_MC = np.random.normal(loc=0, scale=1, size=M_MC)

    Z_IS = np.random.normal(loc=0, scale=1, size=M_IS)
    V_IS = np.random.normal(loc=0, scale=1, size=M_IS)
    S_tau_IS, P_T_IS = data_gen(Z=Z_IS, V=V_IS)


    #define and compile neural network model, setup as in paper
    bi_IS = np.log( np.sum(P_T_IS)/len(P_T_IS))
    model_IS = tf.keras.models.Sequential([
        tf.keras.layers.BatchNormalization(input_shape=(1,)),
        tf.keras.layers.Dense(5, activation='tanh'),
        tf.keras.layers.Dense(1, activation='exponential', bias_initializer=tf.keras.initializers.Constant(value=bi_IS))])
    model_IS.compile(loss='mse', optimizer='adam', metrics=['mse'])
    #train the neural network
    b=model_IS.fit(x=S_tau_IS, y=P_T_IS, epochs=40, batch_size=10000, verbose=0)

    #computation of \theta^*_{NN}
    L_IS = np.column_stack((Z_IS, model_IS.predict(S_tau_IS)[:,0]))
    L_IS = L_IS[L_IS[:,-1].argsort()[::-1]]
    q_alpha_IS_hat = L_IS[int(M_IS*(1-alpha_IS)-1), -1]
    print('q_alpha_IS_hat_NN:', q_alpha_IS_hat)
    
    IS_NN = optimize.least_squares(g_q_alpha_hat_reweighted, x0=np.array([0,1]), bounds=(bnds_lower,bnds_upper), args=(L_IS, q_alpha_IS_hat)).x
    print('IS_NN:', IS_NN)
    #a runtime warning may occur here but in all trial runs this was not of concern as the optimization algorithm still executed successfully and the result was meaningful

    #calculating DT(Z,\theta^*_{NN})
    Z_train_NN = Z_train*IS_NN[1] + IS_NN[0]
    Z_val_NN = Z_val*IS_NN[1] + IS_NN[0]
    Z_test_NN = Z_test*IS_NN[1] + IS_NN[0]
    Z_MC_NN = Z_MC*IS_NN[1] + IS_NN[0]
    #calculating the risk factors under the IS distribution and corresponding option prices
    S_tau_train_NN, P_T_train_NN = data_gen(Z=Z_train_NN, V=V_train)
    S_tau_val_NN, P_T_val_NN = data_gen(Z=Z_val_NN, V=V_val)
    S_tau_test_NN, P_T_test_NN = data_gen(Z=Z_test_NN, V=V_test)
    S_tau_MC_NN, P_T_MC_NN = data_gen(Z=Z_MC_NN, V=V_MC)

    #define and compile neural network model, setup as in paper
    bi = np.log( np.sum(P_T_train_NN)/len(P_T_train_NN))
    model = tf.keras.models.Sequential([
        tf.keras.layers.BatchNormalization(input_shape=(1,)),
        tf.keras.layers.Dense(5, activation='tanh'),
        tf.keras.layers.Dense(1, activation='exponential', bias_initializer=tf.keras.initializers.Constant(value=bi))])
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    #train the model
    hist = model.fit(x=S_tau_train_NN, y=P_T_train_NN, epochs=40, batch_size=10000, validation_data=(S_tau_val_NN,P_T_val_NN), verbose=0)

    #Calculating the parts of the test set that fall into B_1 and B_2
    s_40 = s_0 * np.exp( (mu-0.5*sigma**2)*tau + sigma*np.sqrt(tau)*stats.norm.ppf(0.4, loc=0, scale=1))
    s_70 = s_0 * np.exp( (mu-0.5*sigma**2)*tau + sigma*np.sqrt(tau)*stats.norm.ppf(0.7, loc=0, scale=1))
    B_1_NN = S_tau_test_NN < s_40
    B_2_NN = S_tau_test_NN > s_70

    #computation of the metrics (a), (b), (c) with B_1 and (c) with B_2 for the neural network
    P_T_pred_NN = model.predict(S_tau_test_NN)[:,0]
    mse_train_NN = hist.history['mse'][-1]
    mse_val_NN = hist.history['val_mse'][-1]
    mc_tmp = P_T_pred_NN - P_T_test_NN
    metric_a_NN = np.sum(mc_tmp)/len(P_T_test_NN)
    metric_b_NN = np.sum((mc_tmp)*P_T_pred_NN)/len(P_T_test_NN)
    metric_c_B_1_NN = np.sum(mc_tmp[B_1_NN])/len(P_T_test_NN)
    metric_c_B_2_NN = np.sum(mc_tmp[B_2_NN])/len(P_T_test_NN)
    
    #computation of GlueVaR using the neural network
    P_T_pred_MC_NN = model.predict(S_tau_MC_NN)[:,0]
    MC = np.column_stack((Z_MC_NN, P_T_pred_MC_NN))
    MC_sort = MC[MC[:,-1].argsort()[::-1]]
    w = f(MC_sort[:,0])/(M_MC*f_theta(y=MC_sort[:,0], x=IS_NN))

    GlueVaR_hat_NN = GlueVaR_IS(omega=omega_Glue, L=MC_sort[:,-1], alpha=alpha_Glue, beta=beta_Glue, w=w)
    print('GlueVaR_hat_NN:',GlueVaR_hat_NN)



    #define and train random forest with the same hyperparameter as in Put_GlueVaR_IS_250.ipynb
    rfr_IS = RandomForestRegressor(n_estimators=160, criterion='squared_error', min_samples_leaf=250, bootstrap=True, verbose=0, n_jobs=-1)
    rfr_IS.fit(X=S_tau_IS.reshape(-1,1), y=P_T_IS)

    #computation of \theta^*_{RF}
    L_IS = np.column_stack((Z_IS, rfr_IS.predict(S_tau_IS.reshape(-1,1))))
    L_IS = L_IS[L_IS[:,-1].argsort()[::-1]]
    q_alpha_IS_hat = L_IS[int(M_IS*(1-alpha_IS)-1), -1]
    print('q_alpha_IS_hat_RF:', q_alpha_IS_hat)
    
    IS_RF = optimize.least_squares(g_q_alpha_hat_reweighted, x0=np.array([0,1]), bounds=(bnds_lower,bnds_upper), args=(L_IS, q_alpha_IS_hat)).x
    print('IS_RF:', IS_RF)
    #a runtime warning may occur here but in all trial runs this was not of concern as the optimization algorithm still executed successfully and the result was meaningful

    #calculating DT(Z,\theta^*_{RF})
    Z_train_RF = Z_train*IS_RF[1] + IS_RF[0]
    Z_val_RF = Z_val*IS_RF[1] + IS_RF[0]
    Z_test_RF = Z_test*IS_RF[1] + IS_RF[0]
    Z_MC_RF = Z_MC*IS_RF[1] + IS_RF[0]
    #calculating the risk factors under the IS distribution and corresponding option prices
    S_tau_train_RF, P_T_train_RF = data_gen(Z=Z_train_RF, V=V_train)
    S_tau_val_RF, P_T_val_RF = data_gen(Z=Z_val_RF, V=V_val)
    S_tau_test_RF, P_T_test_RF = data_gen(Z=Z_test_RF, V=V_test)
    S_tau_MC_RF, P_T_MC_RF = data_gen(Z=Z_MC_RF, V=V_MC)
    
    #hypyerparameter tuning with orientation around the value found in Put_GlueVaR_IS_250.ipynb
    min_samples_leaf_list = [4900,5100,5300]
    opt_param = 0
    opt_score = np.inf

    for min_samples_leaf in min_samples_leaf_list:
        rfr_tuning = RandomForestRegressor(n_estimators=160, min_samples_leaf=min_samples_leaf, bootstrap=True, criterion='squared_error', verbose=0, n_jobs=-1)
        rfr_tuning.fit(X=S_tau_train_RF.reshape(-1,1), y=P_T_train_RF)
        score = mean_squared_error(y_true=P_T_val_RF, y_pred=rfr_tuning.predict(S_tau_val_RF.reshape(-1,1)))
        if score < opt_score:
            opt_param = min_samples_leaf
            opt_score = score

    #definition and training of a random forest with hyperparameters found through grid search
    rfr = RandomForestRegressor(n_estimators=400, criterion='squared_error', min_samples_leaf=int(opt_param), bootstrap=True, verbose=0, warm_start=True, n_jobs=-1)
    rfr.fit(X=S_tau_train_RF.reshape(-1,1), y=P_T_train_RF)

    #Calculating the parts of the test set that fall into B_1 and B_2
    s_40 = s_0 * np.exp( (mu-0.5*sigma**2)*tau + sigma*np.sqrt(tau)*stats.norm.ppf(0.4, loc=0, scale=1))
    s_70 = s_0 * np.exp( (mu-0.5*sigma**2)*tau + sigma*np.sqrt(tau)*stats.norm.ppf(0.7, loc=0, scale=1))
    B_1_RF = S_tau_test_RF < s_40
    B_2_RF = S_tau_test_RF > s_70

    #computation of the metrics (a), (b), (c) with B_1 and (c) with B_2 and training/validation MSE for the random forest
    mse_train_RF = mean_squared_error(y_pred=rfr.predict(S_tau_train_RF.reshape(-1,1)),y_true=P_T_train_RF)
    P_T_pred_RF = rfr.predict(S_tau_test_RF.reshape(-1,1))
    mse_val_RF = mean_squared_error(y_pred=P_T_pred_RF,y_true=P_T_test_RF)
    mc_tmp = P_T_pred_RF - P_T_test_RF
    metric_a_RF = np.sum(mc_tmp)/len(P_T_test_RF)
    metric_b_RF = np.sum((mc_tmp)*P_T_pred_RF)/len(P_T_test_RF)
    metric_c_B_1_RF = np.sum(mc_tmp[B_1_RF])/len(P_T_test_RF)
    metric_c_B_2_RF = np.sum(mc_tmp[B_2_RF])/len(P_T_test_RF)

    #computation of GlueVaR using the neural network
    P_T_pred_MC_RF = rfr.predict(S_tau_MC_RF.reshape(-1,1))
    MC = np.column_stack((Z_MC_RF, P_T_pred_MC_RF))
    MC_sort = MC[MC[:,-1].argsort()[::-1]]
    w = f(MC_sort[:,0])/(M_MC*f_theta(y=MC_sort[:,0], x=IS_RF))

    GlueVaR_hat_RF = GlueVaR_IS(omega=omega_Glue, L=MC_sort[:,-1], alpha=alpha_Glue, beta=beta_Glue, w=w)
    print('GlueVaR_hat_RF:',GlueVaR_hat_RF)


    #save results for further evaluation
    output = np.array([[mse_train_NN,mse_val_NN,metric_a_NN,metric_b_NN,metric_c_B_1_NN,metric_c_B_2_NN,IS_NN[0],IS_NN[1],GlueVaR_hat_NN],
                      [mse_train_RF,mse_val_RF,metric_a_RF,metric_b_RF,metric_c_B_1_RF,metric_c_B_2_RF,IS_RF[0],IS_RF[1],GlueVaR_hat_RF]])

    joblib.dump(output,filepath+'output_'+str(run)+'_'+str(j)+'.joblib')
    #prints just for checking while the notebook is running
    print(j)

2021-11-08 08:52:05.493176: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-08 08:52:07.574457: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30988 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:15:00.0, compute capability: 7.0
2021-11-08 08:52:07.576052: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30988 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:16:00.0, compute capability: 7.0
2021-11-08 08:52:07.577588: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:

q_alpha_IS_hat_NN: 7.335477828979492
IS_NN: [-2.27176416  0.46655802]
GlueVaR_hat_NN: 7.692127006041471
q_alpha_IS_hat_RF: 7.313580695131586
IS_RF: [-2.10725459  0.50362434]
GlueVaR_hat_RF: 7.67303795269555
0
q_alpha_IS_hat_NN: 7.2439188957214355
IS_NN: [-2.27587451  0.45410072]
GlueVaR_hat_NN: 7.637456009967796
q_alpha_IS_hat_RF: 7.3830771186654705
IS_RF: [-2.12694338  0.48616444]
GlueVaR_hat_RF: 7.681837396935427
1
q_alpha_IS_hat_NN: 7.309138298034668
IS_NN: [-2.27463332  0.47469625]
GlueVaR_hat_NN: 7.640506663336996
q_alpha_IS_hat_RF: 7.472137723417701
IS_RF: [-2.08919628  0.51478248]
GlueVaR_hat_RF: 7.679501318553718
2
q_alpha_IS_hat_NN: 7.314424514770508
IS_NN: [-2.27074188  0.44979601]
GlueVaR_hat_NN: 7.668240596573641
q_alpha_IS_hat_RF: 7.408801243440192
IS_RF: [-2.11432159  0.48717825]
GlueVaR_hat_RF: 7.707340765442007
3
q_alpha_IS_hat_NN: 7.201164722442627
IS_NN: [-2.27453608  0.48437161]
GlueVaR_hat_NN: 7.665252084266866
q_alpha_IS_hat_RF: 7.2976506093163795
IS_RF: [-1.997481

/scratch/slurm_tmpdir/job_20162881/ipykernel_1054751/3861242163.py:15: RuntimeWarning: divide by zero encountered in true_divide
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)
/scratch/slurm_tmpdir/job_20162881/ipykernel_1054751/3861242163.py:15: RuntimeWarning: overflow encountered in true_divide
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)
/scratch/slurm_tmpdir/job_20162881/ipykernel_1054751/3861242163.py:15: RuntimeWarning: invalid value encountered in multiply
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)


IS_NN: [-2.23128812  0.30883577]
GlueVaR_hat_NN: 7.652034341893486
q_alpha_IS_hat_RF: 7.277861260423727
IS_RF: [-2.00369294  0.54774625]
GlueVaR_hat_RF: 7.710067239828593
9
q_alpha_IS_hat_NN: 7.166808128356934
IS_NN: [-2.27917862  0.48036662]
GlueVaR_hat_NN: 7.6783651920164235
q_alpha_IS_hat_RF: 7.389378106952786
IS_RF: [-2.05268492  0.52913174]
GlueVaR_hat_RF: 7.713760975814873
10
q_alpha_IS_hat_NN: 7.294952392578125
IS_NN: [-2.28410964  0.43619268]
GlueVaR_hat_NN: 7.690782986022711
q_alpha_IS_hat_RF: 7.480162144639195
IS_RF: [-2.0999074   0.48494493]
GlueVaR_hat_RF: 7.691945869650907
11
q_alpha_IS_hat_NN: 6.681669235229492


/scratch/slurm_tmpdir/job_20162881/ipykernel_1054751/3861242163.py:15: RuntimeWarning: divide by zero encountered in true_divide
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)
/scratch/slurm_tmpdir/job_20162881/ipykernel_1054751/3861242163.py:15: RuntimeWarning: overflow encountered in true_divide
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)
/scratch/slurm_tmpdir/job_20162881/ipykernel_1054751/3861242163.py:15: RuntimeWarning: invalid value encountered in multiply
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)


...

IS_NN: [-1.24663698  0.15594689]
GlueVaR_hat_NN: 6.833640499772063
q_alpha_IS_hat_RF: 7.449459792957306
IS_RF: [-2.13695819  0.48934917]
GlueVaR_hat_RF: 7.708035624905286
18
q_alpha_IS_hat_NN: 7.137059211730957
IS_NN: [-2.27573924  0.46033355]
GlueVaR_hat_NN: 7.682342395903037
q_alpha_IS_hat_RF: 7.280096327442786
IS_RF: [-2.04023359  0.51847815]
GlueVaR_hat_RF: 7.679336952821183
19
q_alpha_IS_hat_NN: 6.892298221588135
IS_NN: [-2.27137448  0.44391938]
GlueVaR_hat_NN: 7.68334564140062
q_alpha_IS_hat_RF: 7.4864260979243324
IS_RF: [-2.09561225  0.4926611 ]
GlueVaR_hat_RF: 7.676806283508803
20
q_alpha_IS_hat_NN: 7.263342380523682
IS_NN: [-2.27092978  0.46941846]
GlueVaR_hat_NN: 7.701894640072073
q_alpha_IS_hat_RF: 7.411709658837099
IS_RF: [-2.12495801  0.5007394 ]
GlueVaR_hat_RF: 7.711600335791275
21
q_alpha_IS_hat_NN: 7.034316062927246
IS_NN: [-2.27108594  0.4420101 ]
GlueVaR_hat_NN: 7.642546955569854
q_alpha_IS_hat_RF: 7.447248729419142
IS_RF: [-2.00824037  0.51540419]
GlueVaR_hat_RF: 7.6

/scratch/slurm_tmpdir/job_20162881/ipykernel_1054751/3861242163.py:15: RuntimeWarning: divide by zero encountered in true_divide
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)
/scratch/slurm_tmpdir/job_20162881/ipykernel_1054751/3861242163.py:15: RuntimeWarning: overflow encountered in true_divide
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)
/scratch/slurm_tmpdir/job_20162881/ipykernel_1054751/3861242163.py:15: RuntimeWarning: invalid value encountered in multiply
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)


IS_NN: [-1.55282482  0.16014624]
GlueVaR_hat_NN: 7.081770570143053
q_alpha_IS_hat_RF: 7.379962463546603
IS_RF: [-2.11922312  0.47877785]
GlueVaR_hat_RF: 7.672637003711467
59
q_alpha_IS_hat_NN: 7.154638290405273
IS_NN: [-2.27460236  0.46093723]
GlueVaR_hat_NN: 7.683960625922248
q_alpha_IS_hat_RF: 7.192182886162291
IS_RF: [-2.01642019  0.52447464]
GlueVaR_hat_RF: 7.67790768835917
60
q_alpha_IS_hat_NN: 6.892168998718262
IS_NN: [-2.27768924  0.43166829]
GlueVaR_hat_NN: 7.695864146751495
q_alpha_IS_hat_RF: 7.31047792660244
IS_RF: [-2.05340312  0.53965284]
GlueVaR_hat_RF: 7.686565770940751
61
q_alpha_IS_hat_NN: 7.277825832366943
IS_NN: [-2.28512908  0.46597585]
GlueVaR_hat_NN: 7.638041402742649
q_alpha_IS_hat_RF: 7.2721119105726135
IS_RF: [-2.01649646  0.53027827]
GlueVaR_hat_RF: 7.705139970676328
62
q_alpha_IS_hat_NN: 7.298981666564941
IS_NN: [-2.27487884  0.43490979]
GlueVaR_hat_NN: 7.682256060477725
q_alpha_IS_hat_RF: 7.4218938005109525
IS_RF: [-2.13415821  0.47197753]
GlueVaR_hat_RF: 7.6

/scratch/slurm_tmpdir/job_20162881/ipykernel_1054751/3861242163.py:15: RuntimeWarning: divide by zero encountered in true_divide
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)
/scratch/slurm_tmpdir/job_20162881/ipykernel_1054751/3861242163.py:15: RuntimeWarning: overflow encountered in true_divide
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)
/scratch/slurm_tmpdir/job_20162881/ipykernel_1054751/3861242163.py:15: RuntimeWarning: invalid value encountered in multiply
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)


IS_NN: [-1.49680353  0.16363389]
GlueVaR_hat_NN: 7.162548208812087
q_alpha_IS_hat_RF: 7.346538674625532
IS_RF: [-2.01182205  0.53045546]
GlueVaR_hat_RF: 7.6734495991474265
76
q_alpha_IS_hat_NN: 8.21049690246582
IS_NN: [-2.28131481  0.45317511]
GlueVaR_hat_NN: 7.715087026091506
q_alpha_IS_hat_RF: 7.48607352129242
IS_RF: [-2.05508028  0.5065012 ]
GlueVaR_hat_RF: 7.704242484251061
77
q_alpha_IS_hat_NN: 7.05504035949707
IS_NN: [-2.27129075  0.44805396]
GlueVaR_hat_NN: 7.710881647291561
q_alpha_IS_hat_RF: 7.328709966339434
IS_RF: [-2.16592095  0.47428786]
GlueVaR_hat_RF: 7.694983657086367
78
q_alpha_IS_hat_NN: 7.264336109161377
IS_NN: [-2.26652153  0.47612301]
GlueVaR_hat_NN: 7.660013310746496
q_alpha_IS_hat_RF: 7.444942894006958
IS_RF: [-2.0480627   0.52454107]
GlueVaR_hat_RF: 7.678041489854483
79
q_alpha_IS_hat_NN: 7.330134868621826
IS_NN: [-2.27705222  0.44444832]
GlueVaR_hat_NN: 7.698597005620467
q_alpha_IS_hat_RF: 7.3719295192824745
IS_RF: [-2.09787124  0.49046756]
GlueVaR_hat_RF: 7.70